# MULTIPLE - Hybrid Spectral Net

In [10]:
%%capture
!python -m pip install -U abraia
!python -m pip install wget

import os

if not os.getenv('ABRAIA_KEY'):
   #@markdown <a href="https://abraia.me/console/settings" target="_blank">Get your ABRAIA_KEY</a>
   abraia_key = ''  #@param {type: "string"}
   %env ABRAIA_KEY=$abraia_key

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from abraia import Multiple, hsi

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100
 
multiple = Multiple()

## Data loading

In [11]:
dataset = 'IP'
X, y, class_names = hsi.load_dataset(dataset)
X.shape, y.shape

((145, 145, 200), (145, 145))

In [12]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    newX[margin:X.shape[0] + margin, margin:X.shape[1] + margin, :] = X
    return newX

def patch(data, height_index, width_index, patch_size):
    height_slice = slice(height_index, height_index + patch_size)
    width_slice = slice(width_index, width_index + patch_size)
    return data[height_slice, width_slice, :]

def create_patches(X, patch_size):
    patches = []
    width, height = X.shape[1], X.shape[0]
    X = padWithZeros(X, patch_size // 2)
    for i in range(height):
        for j in range(width):
            image_patch = patch(X, i, j, patch_size)
            patches.append(image_patch.reshape(image_patch.shape + (1,)).astype('float32'))
    return np.array(patches)

def create_image_cubes(X, y, patch_size):
    width, height = X.shape[1], X.shape[0]
    patchesData = create_patches(X, patch_size)
    labels = []
    for i in range(height):
        for j in range(width):
            labels.append(y[i, j])
    patchesLabels = np.array(labels)
    return patchesData, patchesLabels

def generate_training_data(X, y, window_size, K, train_ratio=0.3):
    X = hsi.principal_components(X, n_components=K)
    X, y = create_image_cubes(X, y, window_size)
    X_train, X_test, y_train, y_test = hsi.split_train_test(X, y, train_ratio)
    X_train = X_train.reshape(-1, window_size, window_size, K, 1)
    X_test = X_test.reshape(-1, window_size, window_size, K, 1)
    return X_train, X_test, y_train, y_test

In [13]:
patch_size = 25
K = 30 if dataset == 'IP' else 15
X_train, X_test, y_train, y_test = generate_training_data(X, y, patch_size, K, train_ratio=0.3)
X_train.shape, y_train.shape

## Model training and validation

In [1]:
from tensorflow import keras
from keras.models import Model
from keras.utils import np_utils
from keras.layers import Input, Conv2D, Conv3D, Flatten, Dense, Reshape, Dropout

def create_model(window_size, n_bands, output_units):
    ## input layer
    input_layer = Input((window_size, window_size, n_bands, 1))
    ## convolutional layers
    conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 7), activation='relu')(input_layer)
    conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 5), activation='relu')(conv_layer1)
    conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(conv_layer2)
    conv_layer3 = Reshape((conv_layer3.shape[1], conv_layer3.shape[2], conv_layer3.shape[3] * conv_layer3.shape[4]))(conv_layer3)
    conv_layer4 = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(conv_layer3)
    flatten_layer = Flatten()(conv_layer4)
    ## fully connected layers
    dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
    dense_layer1 = Dropout(0.4)(dense_layer1)
    dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
    dense_layer2 = Dropout(0.4)(dense_layer2)
    output_layer = Dense(units=output_units, activation='softmax')(dense_layer2)
    # define the model with input layer and output layer
    model = Model(inputs=input_layer, outputs=output_layer)
    # compiling the model
    adam = keras.optimizers.Adam(learning_rate=0.001, decay=1e-06)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

def train_model(model, X_train, y_train, batch_size=256, epochs=50):
    history = model.fit(x=X_train, y=np_utils.to_categorical(y_train), batch_size=batch_size, epochs=epochs)
    return history

def evaluate_model(model, X_test, y_test):
    return np.argmax(model.predict(X_test), axis=1)

def predict_model(model, X, patch_size, K):
    width, height = X.shape[1], X.shape[0]
    X = hsi.principal_components(X, n_components=K)
    X_pred = create_patches(X, patch_size)
    y_pred = np.argmax(model.predict(X_pred), axis=1)
    output = np.zeros((height, width))
    for i in range(height):
        for j in range(width):
            k = i * width + j
            output[i, j] = y_pred[k]
    return output.astype(int)

def plot_train_history(history):
    plt.ylim(0, 1.01)
    plt.grid()
    plt.plot(history.history['loss'])
    plt.plot(history.history['accuracy'])
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend(['Training loss','Test accuracy'], loc='upper right')

2021-08-24 12:14:00.506759: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-24 12:14:00.506817: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model = create_model(patch_size, K, len(class_names))
history = train_model(model, X_train, y_train)
plot_train_history(history)

NameError: name 'class_names' is not defined

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = evaluate_model(model, X_test, y_test)

print('Overall accuracy', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=class_names))
print(confusion_matrix(y_test, y_pred))

## Classification results

In [ ]:
output = predict_model(model, X, patch_size, K)

In [1]:
plt.subplot(121)
plt.title('Ground truth')
plt.imshow(y, cmap='nipy_spectral')
plt.axis('off')
plt.subplot(122)
plt.title('Classification')
plt.imshow(output, cmap='nipy_spectral')
plt.axis('off')

NameError: name 'plt' is not defined